In [33]:
# %pip install torch-xla==2.4.0
# %pip uninstall torch -y
# %pip install torch==2.5.1
# %pip uninstall torchvision -y
# %pip install torchvision

In [34]:
import warnings
warnings.filterwarnings("ignore")
import torch
import os, sys
import random
import cv2
from collections import defaultdict
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
import math

from PIL import Image
import time
from keras.utils import Progbar
from tqdm import tqdm
import logging


In [35]:
print(sys.path)

['d:\\all\\all project\\Augmentation_project\\env\\python310.zip', 'd:\\all\\all project\\Augmentation_project\\env\\DLLs', 'd:\\all\\all project\\Augmentation_project\\env\\lib', 'd:\\all\\all project\\Augmentation_project\\env', '', 'd:\\all\\all project\\Augmentation_project\\env\\lib\\site-packages', 'd:\\all\\all project\\Augmentation_project\\env\\lib\\site-packages\\win32', 'd:\\all\\all project\\Augmentation_project\\env\\lib\\site-packages\\win32\\lib', 'd:\\all\\all project\\Augmentation_project\\env\\lib\\site-packages\\Pythonwin', 'd:\\all\\all project\\Augmentation_project\\env\\lib\\site-packages\\setuptools\\_vendor', 'd:\\all\\all project\\Quanvolution proj\\scripts']


In [36]:
new_path = os.path.abspath("../scripts")

if new_path not in sys.path:
    sys.path.append(new_path)
#from my_module import some_function

In [37]:
from QuanSaliencyMix import QuanSaliencyMix
from SaliencyMix import SaliencyMix
from evaluate import evaluate
from resnet18 import build_resnet18
from WarmupCosineLR import WarmupCosineLR

In [38]:
def check (x):
    print (type(x), x.shape)

In [39]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label, idx 

In [40]:
data_path="D:\\all\\all project\\Quanvolution proj\\data\\sub_processed_CIFAR_10"
aug_path="D:\\all\\all project\\Quanvolution proj\\data\\sub_transformed_dataset.pt"
device='cpu'
resize_size=(128, 128)
split_ratio=(0.8, 0.2)
normalize=True
num_workers=0
nor_mean=(0.4914, 0.4822, 0.4465)
nor_std=(0.2471, 0.2435, 0.2616)
random_state=0
batch_size=32
# epochs=10
# optimizer=lambda params: optim.SGD(params, lr=1e-2, weight_decay=1e-2, momentum=0.9,nesterov=True)
# scheduler=lambda optimizer: WarmupCosineLR(optimizer=optimizer, warmup_epochs=30, max_epochs=100, warmup_start_lr=1e-8, eta_min=1e-8)

In [41]:
dataset = datasets.ImageFolder(root=data_path,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Resize(resize_size),
                                   transforms.Normalize(mean=nor_mean, std=nor_std)
                               ]))
images, targets = [], []
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for img_batch, label_batch in loader:
    images.append(img_batch)
    targets.append(label_batch)

images = torch.cat(images, dim=0)
targets = torch.cat(targets, dim=0)

custom_dataset = CustomDataset(data=images, labels=targets)

train_size = int(split_ratio[0] * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(
            custom_dataset, [train_size, test_size],
            generator=torch.Generator().manual_seed(random_state)
        )
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, generator=torch.Generator().manual_seed(random_state)
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

dataset_augmented = torch.load(aug_path)

In [42]:
print (type(dataset_augmented))

<class 'list'>


In [43]:
train_size = int(split_ratio[0] * len(dataset))
test_size = len(dataset) - train_size

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, generator=torch.Generator().manual_seed(random_state)
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

In [44]:
def train(model, 
          epochs=1,
          beta=1, 
          p=0.5, 
          is_quantum=False,
          criterion=getattr(nn, "CrossEntropyLoss")(), 
          optimizer=lambda params: optim.SGD(params, lr=1e-2, weight_decay=1e-2, momentum=0.9,nesterov=True), 
          scheduler=lambda optimizer: WarmupCosineLR(optimizer=optimizer, warmup_epochs=30, max_epochs=100, warmup_start_lr=1e-8, eta_min=1e-8)):
    history = defaultdict(list)
    np.random.seed(random_state)

    optimizer = optimizer(model.parameters())
    scheduler = scheduler(optimizer) if scheduler else None
    best_val_accuracy = 0.0
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        running_loss, correct_samples, total_samples = 0.0, 0, 0
        progbar = Progbar(target=len(train_loader), verbose=1)
        
        for batch_idx, (inputs, labels, idxs) in enumerate(train_loader):
            if np.random.rand(1) < p: 
                sub_augmented = []
                for _ in idxs:
                    sub_augmented.extend(dataset_augmented[_*4:_*4+4])
                # print(type(sub_augmented[0]), sub_augmented[0].shape)
                sub_augmented = torch.stack(sub_augmented)   
                inputs, label_a, label_b, lam = QuanSaliencyMix(inputs, sub_augmented, labels, beta) if is_quantum==True else SaliencyMix(inputs, labels, beta) 
                inputs, label_a, label_b = inputs.to(device), label_a.to(device), label_b.to(device)
                labels = label_a
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, label_a) * lam + criterion(outputs, label_b) * (1 - lam) 
            else:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            correct_samples += (outputs.argmax(dim=1) == labels).sum().item()
            total_samples += labels.size(0)

            avg_train_loss = running_loss / (batch_idx + 1)
            avg_train_accuracy = correct_samples / total_samples
            progbar.update(batch_idx + 1, values=[("loss", avg_train_loss), ("accuracy", avg_train_accuracy)])
        
        if scheduler:
            scheduler.step()

        val_loss, val_accuracy = evaluate(criterion=criterion, model=model, loader=test_loader, device=device)
        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(avg_train_accuracy)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)
        print(f"Epoch {epoch + 1} - Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2%}")

        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state_dict = {k: v.clone() for k, v in model.state_dict().items()}
            print("Best model weights saved.")
    model.load_state_dict(best_model_state_dict)
    print ("Map checking: ", evaluate(criterion=criterion, model=model, loader=test_loader, device=device))
    return model, best_model_state_dict, history

In [45]:
model1 = build_resnet18()
model1 = model1.to(device)

In [ ]:
# train(model=model1, epochs=2, p=0)

Epoch 1/2
25/25 ━━━━━━━━━━━━━━━━━━━━ 514s 21s/step - loss: 2.3703 - accuracy: 0.1052
Epoch 1 - Validation Loss: 2.3155, Accuracy: 5.50%
Best model weights saved.
Epoch 2/2
25/25 ━━━━━━━━━━━━━━━━━━━━ 555s 22s/step - loss: 2.3199 - accuracy: 0.1263
Epoch 2 - Validation Loss: 2.2433, Accuracy: 16.50%
Best model weights saved.
Map checking:  (2.243336238861084, 0.165)


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=

In [47]:
model2 = build_resnet18()
model2 = model2.to(device)

In [48]:
train(model=model2, epochs=2, p=0.5, is_quantum=False) #standard saliency

Epoch 1/2
 4/25 ━━━━━━━━━━━━━━━━━━━━ 7:22 21s/step - loss: 2.2622 - accuracy: 0.2285

TypeError: 'module' object is not callable

In [ ]:
model3 = build_resnet18()
model3 = model3.to(device)

In [ ]:
train(model=model3, epochs=2, p=0.5, is_quantum=True)